# Importar librerias

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib as plt
import matplotlib.pyplot as plt
#import rpy2.robjects as robjects
import statsmodels.api as sm
from statsmodels.formula.api import ols
from astsadata import *
import warnings


## 1. Cargar datos

In [ ]:
%%time
data_full = pd.read_csv("time_series_60min_singleindex.csv")
data = data_full[['utc_timestamp','AT_load_actual_entsoe_transparency']][40000:].copy()
data.head()

In [ ]:
data.shape

1.1. Formato correcto

In [ ]:
data[['utc_timestamp']] = data[['utc_timestamp']].apply(pd.to_datetime)
data['local_timestamp'] = data['utc_timestamp'].dt.tz_convert('Europe/Vienna')
data.head()

1.2. Quitar datos nulos

In [ ]:
data = data.dropna()
data.shape

1.3. Convertir la fecha en el index de la serie

In [ ]:
data.index = data["local_timestamp"]
data.drop(columns=['utc_timestamp', 'local_timestamp'], inplace=True)
data.head()

## 2. Análisis de datos

### Funciones de soporte para graficar funciones de autocorrelación

In [ ]:
def acf1(x, nlags=None, acf_type="correlation", pacf=False, ax=None, **kwargs):
    lags = np.arange(1, nlags + 1)

    if pacf:
        if acf_type == "correlation":
            values = sm.tsa.pacf(x, nlags=nlags)[1:]
            ylabel = "PACF"
    else:
        if acf_type == "correlation":
            values = sm.tsa.acf(x, nlags=nlags, fft=False)[1:]
            ylabel = "ACF"
        elif acf_type == "covariance":
            values = sm.tsa.acovf(x, nlag=nlags)[1:]
            ylabel = "ACoV"

    if ax is None:
        ax = plt.gca()
 
    ax.bar(lags, values, **kwargs)
    ax.axhline(0, color="black", linewidth=1)
    if acf_type == "correlation":
        conf_level = 1.96 / np.sqrt(x.shape[0])
        ax.axhline(conf_level, color="red", linestyle="--", linewidth=1)
        ax.axhline(-conf_level, color="red", linestyle="--", linewidth=1)
    ax.set_xlabel("LAG")
    ax.set_ylabel(ylabel)

    return ax

In [ ]:
# Graficar serie
data.plot(figsize=(15, 4), linewidth=0.1)

In [ ]:
# ACF y PACF de la serie AT_load_actual_entsoe_transparency
fig, axes = plt.subplots(nrows=2, figsize=(15, 7))

acf1(data, nlags=100, ax=axes[0], width=.5)
axes[0].set_title("ACF: serie AT_load_actual_entsoe_transparency")

acf1(data, nlags=20, pacf=True, ax=axes[1], width=.3)
axes[1].set_title("PACF: serie AT_load_actual_entsoe_transparency")

fig.tight_layout()
plt.show()

## 3. Modelo 

In [ ]:
# , trend="c"
model = sm.tsa.SARIMAX(data, order = (1,1,2),seasonal_order=(2, 1, 2, 24)).fit() # SARIMA(1,0,0)
model.summary()

In [ ]:
fig = model.plot_diagnostics(figsize=(10, 7))
fig.tight_layout()
plt.show()

In [ ]:
model.save("SARIMA11221224.pkl")

## 4. Predicción 

In [ ]:
from pandas.tseries.offsets import DateOffset
pred_date=[data.index[-1] + DateOffset(hours=x)for x in range(0,25)]

In [ ]:
data_pred = data.copy()
data_pred["forecast"] = np.nan
pred_date=pd.DataFrame(index=pred_date[1:],columns=data.columns)

In [ ]:
data_pred=pd.concat([data_pred,pred_date])

In [ ]:
forecast = model.get_prediction(start = len(data), end = len(data)+23)
forecast_mean = forecast.predicted_mean
forecast_U = forecast.predicted_mean + forecast.se_mean
forecast_L = forecast.predicted_mean - forecast.se_mean
forecast_mean.index = pred_date.index
forecast_U.index = pred_date.index
forecast_L.index = pred_date.index

In [ ]:
data_pred['forecast'] = forecast_mean
data_pred['Upper bound'] = forecast_U
data_pred['Lower bound'] = forecast_L

In [ ]:
import plotly.graph_objs as go

data_plot = data_pred.loc['2020-09-28':]
fig = go.Figure([
    go.Scatter(
        name='Demanda energética',
        x=data_plot.index,
        y=data_plot['AT_load_actual_entsoe_transparency'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Proyección',
        x=data_plot.index,
        y=data_plot['forecast'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=data_plot.index,
        y=data_plot['Upper bound'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=data_plot.index,
        y=data_plot['Lower bound'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Wind speed (m/s)',
    title='Continuous, variable value error bars',
    hovermode="x"
)
fig.show()

Para cargar el modelo:

In [ ]:
import statsmodels as sm

In [ ]:
model2 = sm.tsa.statespace.sarimax.SARIMAXResults.load("SARIMA11221224.pkl")